In [ ]:
from django.db import models
from django.contrib.auth.models import User

class UserProfile(models.Model):
    user = models.OneToOneField(User, on_delete=models.CASCADE)
    role = models.CharField(max_length=15, choices=[
        ('admin', 'Admin'),
        ('translator', 'Translator'),
        ('project_manager', 'Project Manager'),
        ('developer', 'Developer'),
        ('client', 'Client'),
    ])
    bio = models.TextField(blank=True)
    profile_picture = models.ImageField(upload_to='profile_pics/', blank=True, null=True)
    phone_number = models.CharField(max_length=15, blank=True, null=True)
    address = models.TextField(blank=True)
    
    def __str__(self):
        return f"{self.user.username} - {self.get_role_display()}"

class Client(models.Model):
    user_profile = models.OneToOneField(UserProfile, on_delete=models.CASCADE)
    secondary_email = models.EmailField(blank=True, null=True)
    secondary_phone_number = models.CharField(max_length=15, blank=True, null=True)
    linkedin_profile = models.URLField(blank=True)
    twitter_handle = models.CharField(max_length=50, blank=True)
    industry = models.CharField(max_length=50, blank=True)
    company_size = models.PositiveIntegerField(null=True, blank=True)
    
    def __str__(self):
        return self.user_profile.user.username

class Interaction(models.Model):
    client = models.ForeignKey(Client, on_delete=models.CASCADE, related_name='interactions')
    date = models.DateTimeField()
    interaction_type = models.CharField(max_length=20, choices=[
        ('meeting', 'Meeting'),
        ('call', 'Call'),
        ('email', 'Email'),
        ('presentation', 'Presentation'),
        ('other', 'Other'),
    ])
    notes = models.TextField()

    def __str__(self):
        return f"{self.client.user_profile.user.username} - {self.get_interaction_type_display()} Interaction"

class Task(models.Model):
    title = models.CharField(max_length=200)
    description = models.TextField()
    assignee = models.ForeignKey(User, on_delete=models.CASCADE, related_name='assigned_tasks')
    due_date = models.DateField()
    status = models.CharField(max_length=20, choices=[
        ('todo', 'To Do'),
        ('in_progress', 'In Progress'),
        ('completed', 'Completed'),
    ], default='todo')
    priority = models.PositiveIntegerField(choices=[
        (1, 'Low'),
        (2, 'Medium'),
        (3, 'High'),
    ], default=2)
    project = models.ForeignKey('Project', on_delete=models.CASCADE, related_name='tasks')
    
    def __str__(self):
        return self.title

class TimeEntry(models.Model):
    user = models.ForeignKey(User, on_delete=models.CASCADE)
    task = models.ForeignKey(Task, on_delete=models.CASCADE, null=True, blank=True)
    project = models.ForeignKey('Project', on_delete=models.CASCADE)
    start_time = models.DateTimeField()
    end_time = models.DateTimeField()
    hours_worked = models.DecimalField(max_digits=5, decimal_places=2)
    
    def __str__(self):
        return f"{self.user.username} - {self.project.title} - {self.start_time} to {self.end_time}"

class Expense(models.Model):
    amount = models.DecimalField(max_digits=10, decimal_places=2)
    category = models.CharField(max_length=50)
    description = models.TextField()
    date = models.DateField()
    project = models.ForeignKey('Project', on_delete=models.CASCADE)

    def __str__(self):
        return f"{self.amount} - {self.category} - {self.project.title} - {self.date}"

class Project(models.Model):
    title = models.CharField(max_length=200)
    description = models.TextField()
    start_date = models.DateField()
    end_date = models.DateField(null=True, blank=True)
    client = models.ForeignKey(Client, on_delete=models.CASCADE, related_name='projects')
    team_members = models.ManyToManyField(User, related_name='projects_assigned')
    budget = models.DecimalField(max_digits=10, decimal_places=2, null=True, blank=True)
    actual_cost = models.DecimalField(max_digits=10, decimal_places=2, null=True, blank=True)
    status = models.CharField(max_length=20, choices=[
        ('pending', 'Pending'),
        ('in_progress', 'In Progress'),
        ('completed', 'Completed'),
    ], default='pending')
    priority = models.PositiveIntegerField(default=1)
    is_billable = models.BooleanField(default=True)
    notes = models.TextField(blank=True)
    
    def __str__(self):
        return self.title


In [ ]:
from django.contrib.auth.models import User
from django.db import models


class UserProfile(models.Model):
    user = models.OneToOneField(User, on_delete=models.CASCADE)
    role = models.CharField(
        max_length=15,
        choices=[
            ("admin", "Admin"),
            ("translator", "Translator"),
            ("project_manager", "Project Manager"),
            ("developer", "Developer"),
            ("client", "Client"),
        ],
    )
    bio = models.TextField(blank=True)
    profile_picture = models.ImageField(
        upload_to="profile_pics/", blank=True, null=True
    )
    phone_number = models.CharField(max_length=15, blank=True, null=True)
    address = models.TextField(blank=True)

    def __str__(self):
        return f"{self.user.username} - {self.get_role_display()}"

    class Meta:
        permissions = [
            ("is_admin", "Can access admin functionalities"),
            ("is_translator", "Can perform translation tasks"),
            ("is_project_manager", "Can manage projects"),
            ("is_developer", "Can perform development tasks"),
            ("is_client", "Is a client"),
        ]


class Client(models.Model):
    user_profile = models.OneToOneField(UserProfile, on_delete=models.CASCADE)
    secondary_email = models.EmailField(blank=True, null=True)
    secondary_phone_number = models.CharField(max_length=15, blank=True, null=True)
    linkedin_profile = models.URLField(blank=True)
    twitter_handle = models.CharField(max_length=50, blank=True)
    industry = models.CharField(max_length=50, blank=True)
    company_size = models.PositiveIntegerField(null=True, blank=True)

    def __str__(self):
        return self.user_profile.user.username


class Interaction(models.Model):
    client = models.ForeignKey(
        Client, on_delete=models.CASCADE, related_name="interactions"
    )
    date = models.DateTimeField()
    interaction_type = models.CharField(
        max_length=20,
        choices=[
            ("meeting", "Meeting"),
            ("call", "Call"),
            ("email", "Email"),
            ("presentation", "Presentation"),
            ("other", "Other"),
        ],
    )
    notes = models.TextField()

    def __str__(self):
        return f"{self.client.user_profile.user.username} - {self.get_interaction_type_display()} Interaction"


class Task(models.Model):
    title = models.CharField(max_length=200)
    description = models.TextField()
    assignee = models.ForeignKey(
        User, on_delete=models.CASCADE, related_name="assigned_tasks"
    )
    due_date = models.DateField()
    status = models.CharField(
        max_length=20,
        choices=[
            ("todo", "To Do"),
            ("in_progress", "In Progress"),
            ("completed", "Completed"),
        ],
        default="todo",
    )
    priority = models.PositiveIntegerField(
        choices=[
            (1, "Low"),
            (2, "Medium"),
            (3, "High"),
        ],
        default=2,
    )
    project = models.ForeignKey(
        "Project", on_delete=models.CASCADE, related_name="tasks"
    )

    def __str__(self):
        return self.title


class TimeEntry(models.Model):
    user = models.ForeignKey(User, on_delete=models.CASCADE)
    task = models.ForeignKey(Task, on_delete=models.CASCADE, null=True, blank=True)
    project = models.ForeignKey("Project", on_delete=models.CASCADE)
    start_time = models.DateTimeField()
    end_time = models.DateTimeField()
    hours_worked = models.DecimalField(max_digits=5, decimal_places=2)

    def __str__(self):
        return f"{self.user.username} - {self.project.title} - {self.start_time} to {self.end_time}"


class Expense(models.Model):
    amount = models.DecimalField(max_digits=10, decimal_places=2)
    category = models.CharField(max_length=50)
    description = models.TextField()
    date = models.DateField()
    project = models.ForeignKey("Project", on_delete=models.CASCADE)

    def __str__(self):
        return f"{self.amount} - {self.category} - {self.project.title} - {self.date}"


class Project(models.Model):
    title = models.CharField(max_length=200)
    description = models.TextField()
    start_date = models.DateField()
    end_date = models.DateField(null=True, blank=True)
    client = models.ForeignKey(
        Client, on_delete=models.CASCADE, related_name="projects"
    )
    team_members = models.ManyToManyField(User, related_name="projects_assigned")
    budget = models.DecimalField(max_digits=10, decimal_places=2, null=True, blank=True)
    actual_cost = models.DecimalField(
        max_digits=10, decimal_places=2, null=True, blank=True
    )
    status = models.CharField(
        max_length=20,
        choices=[
            ("pending", "Pending"),
            ("in_progress", "In Progress"),
            ("completed", "Completed"),
        ],
        default="pending",
    )
    priority = models.PositiveIntegerField(default=1)
    is_billable = models.BooleanField(default=True)
    notes = models.TextField(blank=True)

    def __str__(self):
        return self.title


# Assigning roles to users
user = User.objects.get(username="example_user")
user.userprofile.role = "admin"
user.save()

In [ ]:
from django.contrib.auth.models import AbstractUser, User
from django.db import models
from django.utils.translation import gettext_lazy as _



class CustomUser(AbstractUser):
    # Add custom fields
    role = models.CharField(
        max_length=15,
        choices=[
            ("admin", "Admin"),
            ("translator", "Translator"),
            ("project_manager", "Project Manager"),
            ("developer", "Developer"),
            ("client", "Client"),
        ],
    )
    bio = models.TextField(blank=True)
    profile_picture = models.ImageField(
        upload_to="profile_pics/", blank=True, null=True
    )
    phone_number = models.CharField(max_length=15, blank=True, null=True)
    address = models.TextField(blank=True)


class Role(models.Model):
    """
    Abstract model for roles and permissions.
    """

    ROLE_CHOICES = [
        ("admin", _("Admin")),
        ("translator", _("Translator")),
        ("project_manager", _("Project Manager")),
        ("developer", _("Developer")),
        ("client", _("Client")),
    ]

    role = models.CharField(max_length=15, choices=ROLE_CHOICES)

    class Meta:
        abstract = True


class UserProfile(models.Model):
    """
    Model for user profiles with roles and additional information.
    """

    user = models.OneToOneField(User, on_delete=models.CASCADE)
    bio = models.TextField(blank=True)
    profile_picture = models.ImageField(
        upload_to="profile_pics/", blank=True, null=True
    )
    phone_number = models.CharField(max_length=15, blank=True, null=True)
    address = models.TextField(blank=True)
    role = models.ForeignKey(
        Role, on_delete=models.CASCADE
    )  # Integrate with Role model

    def __str__(self):
        return f"{self.user.username} - {self.role.get_role_display()}"  # Use Role's display name

    class Meta:
        verbose_name = _("User Profile")
        verbose_name_plural = _("User Profiles")

In [ ]:
from django.contrib.auth.models import AbstractUser
from django.db import models
from django.utils.translation import gettext_lazy as _


class Role(models.Model):
    """
    Model for roles in the system.
    """

    name = models.CharField(max_length=15, unique=True)
    description = models.TextField()

    def __str__(self):
        return self.name


class Permission(models.Model):
    """
    Model to represent permissions.
    """

    name = models.CharField(max_length=100, unique=True)
    description = models.TextField()

    def __str__(self):
        return self.name


class CustomUser(AbstractUser):
    """
    Custom user model with additional fields and methods.
    """

    role = models.ForeignKey(Role, on_delete=models.SET_NULL, null=True)
    bio = models.TextField(blank=True)
    profile_picture = models.ImageField(upload_to="profile_pics/", blank=True, null=True)
    phone_number = models.CharField(max_length=15, blank=True, null=True)
    address = models.TextField(blank=True)

    def has_permission(self, permission_name):
        """
        Check if the user's role has the specified permission.
        """
        if self.role:
            return self.role.permissions.filter(name=permission_name).exists()
        return False


class UserProfile(models.Model):
    """
    Model for user profiles with roles and additional information.
    """

    user = models.OneToOneField(CustomUser, on_delete=models.CASCADE)
    bio = models.TextField(blank=True)
    profile_picture = models.ImageField(upload_to="profile_pics/", blank=True, null=True)
    phone_number = models.CharField(max_length=15, blank=True, null=True)
    address = models.TextField(blank=True)
    role = models.ForeignKey(Role, on_delete=models.CASCADE)

    def __str__(self):
        return f"{self.user.username} - {self.role.name}"


In [ ]:
from django.contrib.auth.models import AbstractUser
from django.db import models
from django.utils.translation import gettext_lazy as _


class Permission(models.Model):
    """
    Model to represent permissions.
    """

    name = models.CharField(max_length=100, unique=True)
    description = models.TextField()

    def __str__(self):
        return self.name


class RoleManager(models.Manager):
    def with_permissions(self):
        """
        Returns roles with associated permissions.
        """
        return self.get_queryset().select_related("permissions")


class Role(models.Model):
    """
    Model for roles in the system.
    """

    name = models.CharField(max_length=15, unique=True)
    description = models.TextField()
    permissions = models.ManyToManyField(Permission, related_name="roles")
    objects = RoleManager()

    def __str__(self):
        return self.name


class CustomUser(AbstractUser):
    """
    Custom user model with additional fields and methods.
    """

    role = models.ForeignKey(Role, on_delete=models.SET_NULL, null=True)

    def has_specific_permission(self, permission_name):
        """
        Check if the user has a specific permission.
        """
        return self.user_permissions.filter(name=permission_name).exists() or (
            self.role and self.role.permissions.filter(name=permission_name).exists()
        )


class UserProfile(models.Model):
    """
    Model for user profiles with additional information.
    """

    user = models.OneToOneField(CustomUser, on_delete=models.CASCADE)
    # No redundant fields here

    def __str__(self):
        return self.user.username